Na wstępie pragnę zaznaczyć, że korzystam z tensorflow oraz z feautre columns i estimators. Są one oznaczone jako przestarzałe, jednak wolę pokazać moje szczere rozwiązanie na tych opcjach, zamiast korzystac z np kerasa, którym się aż tak płynnie nie posługuję.

W projekcie rekrutacyjnym chciałbym przedstawić moje rozwiązania dot.problemu titanica, nie mniej nie tego z kaggle'a. Korzystam z nieco prostszego zbioru, w którym nie ma missing ages. 

W moim kodzie w komentarzach mówię krok po kroku o tym, co robię. **Nie rozumiem** w nim natomiast wszystkiego - Część rozwiązań pochodzi z tutoriali oraz stacka. 

Moje rozwiązanie osiaga około 75% skuteczności, ale na pewno nie jest profesjonalnym rozwiązaniem. Wierzę, że po okresie działania w waszym kole, będe mógł wrócić do mojego kodu i zdać sobie sprawę z tego, że teraz potrafię zrobić coś o wiele lepszego.

Mateusz Iwaniuk, IAD (1 semestr)

In [58]:
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x  

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [59]:
df_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # dane do trenowania
df_eval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # dane do testowania

df_eval_2 = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') 

In [60]:
df_train.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [61]:
df_eval.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,35.0,0,0,8.0500,Third,unknown,Southampton,y
1,0,male,54.0,0,0,51.8625,First,E,Southampton,y
2,1,female,58.0,0,0,26.5500,First,C,Southampton,y
3,1,female,55.0,0,0,16.0000,Second,unknown,Southampton,y
4,1,male,34.0,0,0,13.0000,Second,D,Southampton,y


In [62]:
y_train = df_train.pop('survived')
y_eval = df_eval.pop('survived')

In [64]:
#Tworzę feature columns i dzielę dane na kategoryczne i numeryczne
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []

for feature in CATEGORICAL_COLUMNS:
  unique = df_train[feature].unique()
  data = tf.feature_column.categorical_column_with_vocabulary_list(feature,unique)
  feature_columns.append(data)

for feature in NUMERIC_COLUMNS:
  unique = df_train[feature].unique()
  data = tf.feature_column.numeric_column(feature,dtype=tf.float32)
  feature_columns.append(data)




In [65]:
feature_columns

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.strin

In [66]:
#tworzę funkcję pod ewaluację modelu
def make_input_fn(data_df,label_df,num_epochs=10,shuffle=True,batch_size=32): #Nie jestem pewien, czemu domyślnie 32
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df)) #Nie jestem pewien co tu się dzieje
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

In [67]:
#tworzę inputy pod trening i testing
train_input_fn = make_input_fn(df_train, y_train) 
eval_input_fn = make_input_fn(df_eval, y_eval, num_epochs=1, shuffle=False)

In [68]:
#tworzę estimatora bazującego na feature columns
linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [69]:
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

print(result)

{'accuracy': 0.7613636, 'accuracy_baseline': 0.625, 'auc': 0.8270891, 'auc_precision_recall': 0.77352947, 'average_loss': 0.51755387, 'label/mean': 0.375, 'loss': 0.5075015, 'precision': 0.7368421, 'prediction/mean': 0.283961, 'recall': 0.56565654, 'global_step': 200}


In [70]:
result["accuracy"]

0.7613636

In [72]:
#printuję szansę na przezycie w porównaniu z oryginalnymi danymi (df_eval) do testingu
pred_dicts = list(linear_est.predict(eval_input_fn))

for count,pred in enumerate(pred_dicts): #zastąpienie funckji head()
  print(pred)
  if count==10:
    break

probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs

df_eval_2['Survived_chance'] = probs

df_eval_2.head(20)


{'logits': array([-2.9269142], dtype=float32), 'logistic': array([0.05083902], dtype=float32), 'probabilities': array([0.94916105, 0.05083903], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
{'logits': array([-1.9088258], dtype=float32), 'logistic': array([0.12911282], dtype=float32), 'probabilities': array([0.87088716, 0.12911282], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
{'logits': array([-0.00494163], dtype=float32), 'logistic': array([0.4987646], dtype=float32), 'probabilities': array([0.50123537, 0.49876457], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
{'logits': array([-0.22683541], dtype=fl

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone,Survived_chance
0,0,male,35.0,0,0,8.0500,Third,unknown,Southampton,y,0.050839
1,0,male,54.0,0,0,51.8625,First,E,Southampton,y,0.129113
2,1,female,58.0,0,0,26.5500,First,C,Southampton,y,0.498765
3,1,female,55.0,0,0,16.0000,Second,unknown,Southampton,y,0.443533
4,1,male,34.0,0,0,13.0000,Second,D,Southampton,y,0.174764
5,1,female,15.0,0,0,8.0292,Third,unknown,Queenstown,y,0.682797
6,0,female,8.0,3,1,21.0750,Third,unknown,Southampton,n,0.453779
7,0,male,21.0,0,0,8.0500,Third,unknown,Southampton,y,0.080594
8,0,female,18.0,2,0,18.0000,Third,unknown,Southampton,n,0.485153
9,1,female,19.0,0,0,7.8792,Third,unknown,Queenstown,y,0.651837
